Conjuntos de prueba en profundidad

En el ejercicio anterior vimos cómo dividir nuestros datos en conjuntos de entrenamiento y de prueba para evaluar el rendimiento del modelo.

Ahora repetiremos el último ejercicio, pero esta vez veremos qué ocurre cuando dividimos los datos de diferentes formas y proporciones.

Primero recordemos qué hay en nuestro conjunto de datos:

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/dog-training.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/dog-training-switzerland.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
import graphing

data = pandas.read_csv("dog-training.csv", delimiter="\t")

print(f"Dataset shape: {data.shape}")
data.head()

Hagamos una rápida recapitulación de cómo es la distribución de nuestros datos (recordemos que estábamos utilizando peso_último_año para predecir el valor de rescates_último_año).

In [ ]:
# Obtain the label and feature from the original data
dataset = data[['rescues_last_year','weight_last_year']]

# Print the number of observations
print("No. observations:", dataset.shape[0])

# Graph the distribution of variables for the unsplit dataset
graphing.scatter_2D(dataset, 'rescues_last_year', 'weight_last_year')

Observe que se han representado 50 observaciones (que corresponden al número de muestras del conjunto de datos).

Comparación de la proporción de división del conjunto de datos
Ahora vamos a dividir nuestro conjunto de datos utilizando diferentes ratios para entender cómo pueden afectar a nuestros modelos

In [ ]:
from sklearn.model_selection import train_test_split

# Split Dataset using different ratios 50:50, 60:40, 70:30, 80:20
train_5050, test_5050 = train_test_split(dataset, test_size=0.5, random_state=2)
train_6040, test_6040 = train_test_split(dataset, test_size=0.4, random_state=2)
train_7030, test_7030 = train_test_split(dataset, test_size=0.3, random_state=2)
train_8020, test_8020 = train_test_split(dataset, test_size=0.2, random_state=2)

# Add a column to each set to identify if a datapoint belongs to "train" or "test"
train_5050, test_5050 = train_5050.assign(Set="train"), test_5050.assign(Set="test")
train_6040, test_6040 = train_6040.assign(Set="train"), test_6040.assign(Set="test")
train_7030, test_7030 = train_7030.assign(Set="train"), test_7030.assign(Set="test")
train_8020, test_8020 = train_8020.assign(Set="train"), test_8020.assign(Set="test")

# Concatenate the "train" and "test" sets for each split so we can plot them on the same chart
df_5050 = pandas.concat([train_5050, test_5050], axis=0)
df_6040 = pandas.concat([train_6040, test_6040], axis=0)
df_7030 = pandas.concat([train_7030, test_7030], axis=0)
df_8020 = pandas.concat([train_8020, test_8020], axis=0)

# Plot each distribution for comparison
graphing.scatter_2D(df_5050, "weight_last_year", "rescues_last_year", title="50:50 split", label_colour="Set", show=True)
graphing.scatter_2D(df_6040, "weight_last_year", "rescues_last_year", title="60:40 split", label_colour="Set", show=True)
graphing.scatter_2D(df_7030, "weight_last_year", "rescues_last_year", title="70:30 split", label_colour="Set", show=True)
graphing.scatter_2D(df_8020, "weight_last_year", "rescues_last_year", title="80:20 split", label_colour="Set")

Observe cómo las primeras divisiones nos dejan conjuntos de datos de entrenamiento relativamente pequeños. En la división 50:50, sólo tenemos 25 muestras para entrenar.

Por otro lado, las últimas nos dejan muchos más datos para el entrenamiento, pero relativamente pocos para las pruebas. La división 80:20 nos deja sólo 10 muestras en el conjunto de prueba.

Veamos las distribuciones de los datos de entrenamiento para cada división:

In [ ]:
# Add a column for each "train" set that identifies the split used
train_8020 = train_8020.assign(Split="80:20")
train_7030 = train_7030.assign(Split="70:30")
train_6040 = train_6040.assign(Split="60:40")
train_5050 = train_5050.assign(Split="50:50")

# Concatenate training sets so we can plot them on the same chart
split_df = pandas.concat([train_5050, train_6040, train_7030, train_8020], axis=0)

 # Plot a histogram of data distribution
graphing.multiple_histogram(split_df, label_x="rescues_last_year", label_group="Split", title="Distribution of Training data")

Podemos sacar un par de conclusiones del gráfico anterior:

La función train_test_split() que utilizamos hace un trabajo bastante bueno a la hora de mantener una distribución justa de los datos entre las diferentes proporciones. Intenta mantener los diferentes valores representados en la misma proporción.

Sin embargo, cuando utilizamos una proporción 50:50, ¡tenemos que dejar tantos datos fuera del conjunto de entrenamiento que algunos valores ya no están presentes! (¿puedes detectar dónde faltan barras azules?)

El punto 2 es especialmente preocupante, ya que si esos datos no están disponibles para el entrenamiento, nuestro modelo no puede aprender de ellos y, por tanto, no hará predicciones precisas. En otras palabras, ¡utilizar una división 50:50 no parece una buena idea para este conjunto de datos!


Ajuste y evaluación de modelos con diferentes proporciones de división
Vamos a ajustar modelos con diferentes proporciones de división y a ver cómo funcionan:

In [ ]:
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error as mse

def train_and_test_model(name, train, test):
    '''
    This method creates a model, trains it on the provided data, and assesses 
    it against the test data
    '''
    # This formula says that rescues_last_year is explained by weight_last_year
    formula = "rescues_last_year ~ weight_last_year"

    # Create and train a linear regression model using the training data
    model = smf.ols(formula = formula, data = train).fit()

    # Now evaluate the model (by calculating the Mean Squared Error) using the 
    # corresponding "test" set for that split
    correct_answers = test['rescues_last_year']
    predictions = model.predict(test['weight_last_year'])
    MSE = mse(correct_answers, predictions)
    print(name + ' MSE = %f ' % MSE)

    return model


# Train a model and test it for each kind of split
print("Mean Squared Error values (smaller is better)")
model_5050 = train_and_test_model("50:50", train_5050, test_5050)
model_6040 = train_and_test_model("60:40", train_6040, test_6040)
model_7030 = train_and_test_model("70:30", train_7030, test_7030)
model_8020 = train_and_test_model("80:20", train_8020, test_8020)

El código anterior entrena un modelo para cada relación, utilizando el conjunto de entrenamiento correspondiente a esa relación. A continuación, calcula el MSE (error cuadrático medio) de cada modelo utilizando el conjunto de prueba correspondiente.

Los resultados parecen variados. La proporción 80:20 fue la mejor, pero no hay una pauta clara sobre si resulta útil aumentar o reducir el conjunto de entrenamiento.

Hay dos factores que influyen en los resultados. En primer lugar, cuanto mayor sea el conjunto de pruebas, más podemos confiar en los resultados de las pruebas. En segundo lugar, los modelos suelen entrenarse mejor con conjuntos de entrenamiento más grandes.

Estas influencias son contradictorias entre sí, y su influencia se ve exagerada en este caso porque nuestro conjunto de datos es muy pequeño. En esta situación concreta es difícil evaluar si la división 60:40 es realmente mejor que la 70:30, por ejemplo. Esto se debe a que la división 70:30 podría parecer peor porque se probó con un conjunto de pruebas menos representativo (más pequeño).

Evaluación del modelo
Veamos ahora qué ocurre cuando estos modelos se utilizan con un conjunto de datos mucho mayor en el que no se han entrenado ni probado. Esto puede ocurrir en el mundo real porque decidimos retener datos al principio, o simplemente porque recopilamos datos después de entrenar nuestro modelo. En nuestro caso, se trata de nuevos datos que nos ha proporcionado la sección europea de nuestra organización benéfica de rescate en avalanchas.

In [ ]:
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error as mse

# Load some dog statistics from our charity's European arm
swiss_data = pandas.read_csv("dog-training-switzerland.csv", delimiter="\t")

# Show show information about the data
print(f"The Swiss dataset contains {swiss_data.shape[0]} samples")
graphing.scatter_2D(swiss_data, 'rescues_last_year', 'weight_last_year')

Se trata claramente de una muestra de datos mucho mayor. Veamos cómo funcionan nuestros modelos. Ten en cuenta que aquí no estamos volviendo a entrenar los modelos; simplemente los estamos utilizando para hacer predicciones en un nuevo conjunto de datos para evaluar su rendimiento.

In [ ]:
# Test our models against the swiss data
features = swiss_data['weight_last_year']
correct_answers = swiss_data['rescues_last_year']

# We will now assess our models. Notice we're not training them again.
# We are simply testing them against some new data 

# Assess the model trained on a 50:50 split
predictions = model_5050.predict(features)
MSE = mse(correct_answers, predictions)
print('50:50 MSE = %f ' % MSE)

# Assess the model trained on a 60:40 split
predictions = model_6040.predict(features)
MSE = mse(correct_answers, predictions)
print('60:40 MSE = %f ' % MSE)

# Assess the model trained on a 70:30 split
predictions = model_7030.predict(features)
MSE = mse(correct_answers, predictions)
print('70:30 MSE = %f ' % MSE)

# Assess the model trained on a 80:20 split
predictions = model_8020.predict(features)
MSE = mse(correct_answers, predictions)
print('80:20 MSE = %f ' % MSE)

Con este conjunto de datos más amplio, el modelo que utilizó la división 80:20 arrojó el error más bajo y, por tanto, el mejor rendimiento. También se observa un patrón claro: cuanto mayor es el conjunto de datos de entrenamiento, mejor es el rendimiento del modelo tras el entrenamiento.

En conjunto, esto demuestra que deberíamos probar y evaluar diferentes divisiones de entrenamiento/prueba al construir modelos de aprendizaje automático, y que generalmente las divisiones que favorecen el conjunto de entrenamiento con más datos producirán mejores resultados.

Resumen
En este ejercicio has aprendido los siguientes conceptos:

Puede utilizar diferentes proporciones al dividir su conjunto de datos en conjuntos de entrenamiento y prueba.<br>
Diferentes proporciones producen diferentes distribuciones de variables en los conjuntos de entrenamiento y prueba resultantes.<br>
Cuando las proporciones de entrenamiento:prueba son cercanas, es posible que esté dejando muchos datos fuera del conjunto de entrenamiento, lo que puede tener un impacto negativo en el rendimiento del modelo.<br>
Cuando se construyen modelos, es importante probarlos utilizando diferentes combinaciones de entrenamiento/prueba. Asignar simplemente más datos al conjunto de entrenamiento no siempre garantiza los mejores resultados.